In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import gpflow as gpf
import tensorflow as tf

from gpflow.utilities import print_summary
from gpflow.ci_utils import ci_niter

from reggae.data_loaders import load_barenco_puma, DataHolder

gpf.config.set_default_float(np.float64)
gpf.config.set_default_summary_fmt("notebook")
np.random.seed(0)
%matplotlib inline

MAXITER = ci_niter(2000)
f64 = np.float64
plt.style.use('ggplot')


In [ ]:
m_observed, f_observed, σ2_m_pre, σ2_f_pre, t = load_barenco_puma()
m_df, m_observed = m_observed 
f_df, f_observed = f_observed
# Shape of m_observed = (replicates, genes, times)
m_observed = m_observed
f_observed = f_observed
data_obj = (m_observed, f_observed)

σ2_m_pre = f64(σ2_m_pre)
σ2_f_pre = f64(σ2_f_pre)
noise_data = (σ2_m_pre, σ2_f_pre)


num_genes = m_observed.shape[1]
N_m = m_observed.shape[2]
granularity = 100
τ = np.linspace(0, 12, granularity)
time = (t, τ, None)
data_holder = DataHolder(data_obj, noise_data, time)

Y_var = data_holder.σ2_m_pre[0].reshape(-1)

m_obs = m_observed #(Nr, J, Nm)
m_obs[0, 3] = np.array([2, 2.5, 1.5, 1, 0.5, 0.6, 0.3])



In [ ]:
N = 7  # number of points
D = 1  # number of input dimensions
M = N  # number of inducing points
L = 2  # number of latent GPs
P = m_obs.shape[1]  # number of observations = output dimensions

In [ ]:
# X = np.c_[[np.float64(t) for _ in range(num_genes)]].reshape(-1, 1)
X, Y = data = f64(t[:, None]), m_obs[0].T
Zinit = X#t[:, None] #np.linspace(-5, 5, M)[:, None]
print(X.shape, Y.shape)

# Zinit = np.linspace(-5, 5, M)[:, None]


In [ ]:
print(Zinit)

In [ ]:
def plot_model(m, lower=0, upper=14.0):
    pX = np.linspace(lower, upper, 100)[:, None]
    pY, pYv = m.predict_y(pX)
    if pY.ndim == 3:
        pY = pY[:, 0, :]
    for i in range(Y.shape[1]):        
        plt.plot(X[:,0], Y[:,i], "x", label=f'Gene {i}')
    plt.gca().set_prop_cycle(None)
    plt.plot(pX, pY)
    for i in range(pY.shape[1]):
        top = pY[:, i] + 2.0 * pYv[:, i] ** 0.5
        bot = pY[:, i] - 2.0 * pYv[:, i] ** 0.5
        plt.fill_between(pX[:, 0], top, bot, alpha=0.3)
    plt.xlabel("Time (h)")
    plt.ylabel("Abundance (AU)")
#     plt.title(f"ELBO: {m.elbo(data):.3}")
    plt.plot(Z, Z * 0.0, "|")
    plt.legend()

In [ ]:
def optimize_model_with_scipy(model):
    optimizer = gpf.optimizers.Scipy()
    optimizer.minimize(
        model.training_loss_closure(data),
        variables=model.trainable_variables,
        method="l-bfgs-b",
        options={"disp": True, "maxiter": MAXITER},
    )


In [ ]:
# Create list of kernels for each output
kern_list = [gpf.kernels.SquaredExponential() + gpf.kernels.Linear() for _ in range(L)]
# Create multi-output kernel from kernel list
kernel = gpf.kernels.LinearCoregionalization(
    kern_list, W=np.random.randn(P, L)
)  # Notice that we initialise the mixing matrix W
# initialisation of inducing input locations (M random points from the training inputs)
Z = Zinit.copy()
# create multi-output inducing variables from Z
iv = gpf.inducing_variables.SharedIndependentInducingVariables(
    gpf.inducing_variables.InducingPoints(Z)
)


In [ ]:
q_mu = np.zeros((M, L))
# initialize \sqrt(Σ) of variational posterior to be of shape LxMxM
q_sqrt = np.repeat(np.eye(M)[None, ...], L, axis=0) * 1.0


# create SVGP model as usual and optimize
m = gpf.models.SVGP(
    kernel, gpf.likelihoods.Gaussian(), inducing_variable=iv, q_mu=q_mu, q_sqrt=q_sqrt
)

In [ ]:
optimize_model_with_scipy(m)


In [ ]:
plt.style.use('ggplot')
plt.figure(figsize=(7, 5))
colors = ['orangered', 'slateblue', 'green', 'slategray', 'purple']
plot_model(m)


In [ ]:
# plt.figure(figsize=(7, 3))
plt.imshow(m.kernel.W.numpy(), cmap='gray')
plt.grid(None)
plt.ylabel('Gene')
plt.xlabel('Latent GP')
plt.colorbar()